# 使用BYOM创建Tensorflow Serving的方法

#### 将model上传至S3

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role
import pandas as pd

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name
bucket = sagemaker_session.default_bucket()

In [ ]:
import boto3 

boto3.Session().resource('s3').Bucket(bucket).Object('tf-byom/model.tar.gz').upload_file('model.tar.gz')

#### 不提供inference.py

In [ ]:
# 创建模型，不提供inference.py

from sagemaker.tensorflow import TensorFlowModel

iris_model = TensorFlowModel(model_data='s3://{}/tf-byom/model.tar.gz'.format(bucket),
                        role=role,
                        framework_version='1.15.2'
                        )

In [ ]:
# 部署

predictor = iris_model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

In [ ]:
# 将iris_test.csv数据中的前五行数据取出，构造classify接口所能接受的examples

df = pd.read_csv('iris_test.csv', index_col=False)
examples = []
for i in df.index[:5]:
    row = {col: df.loc[i, col] for col in df.columns[:4]}
    examples.append(row)
data1 = {'examples': examples}
data1

In [ ]:
# 推理

predictor.classify(data1)

#### 提供inference.py

In [ ]:
# 创建模型，提供inference.py，其中写好了数据的前处理（将列表转化为classify接口所能接受的数据格式）

from sagemaker.tensorflow import TensorFlowModel

iris_model_with_infer = TensorFlowModel(model_data='s3://{}/tf-byom/model.tar.gz'.format(bucket),
                        role=role,
                        framework_version='1.15.2',
                        entry_point='inference.py'
                        )

In [ ]:
# 默认的serializer为json，且classify api仅支持json serializer

predictor_with_infer = iris_model_with_infer.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

In [ ]:
# 将iris_test.csv数据中的前五行数据取出，构造成列表

data2 = df.iloc[:5, :4].values.tolist()
data2

In [ ]:
# 推理

predictor_with_infer.classify(data2)